This Script check the missing data imputation methods for Train edss and Train static data. Four methods including
K-means, KNN, missforest and micefirest are considered for analysis. 

In [4]:
!pip install miceforest
https://www.analyticsvidhya.com/blog/2022/05/handling-missing-values-with-random-forest/
https://ishanjainoffical.medium.com/missing-value-treatment-advanced-methods-f7fa05ec0f39

     ---------------------------------------- 58.2/58.2 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import miceforest as mf
import random
import sklearn.neighbors._base 
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

In [3]:
completed_data = pd.read_csv("Train_edss_static_completed.csv")
missing_data = pd.read_csv("Train_edss_static_with_missing_0.1.csv")

In [4]:
completed_data = completed_data.iloc[:,1:]
missing_data = missing_data.iloc[:,1:]

In [7]:
# K-means imputation
kmean_completed = completed_data.copy()
kmean_missing = missing_data.copy()

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmean_completed['Cluster'] = kmeans.fit_predict(kmean_completed) 
kmean_missing['Cluster'] = kmean_completed['Cluster']

for cluster_label in range(n_clusters): 
    cluster_subset = kmean_completed[kmean_completed['Cluster'] == cluster_label]
    cluster_mean = cluster_subset[kmean_completed.columns[11:19]].mean().round()
    filled_data = kmean_missing.loc[kmean_missing['Cluster'] == cluster_label,
                                    kmean_missing.columns[11:19]].fillna(cluster_mean, inplace=False)
    kmean_missing.update(filled_data)
    
kmean_missing.drop(["Cluster"], axis=1, inplace = True)
kmean_missing.head()

,week_from_baseline,edss_as_evaluated_by_clinician,sex,ms_in_pediatric_age,age_at_onset,age_at_baseline,spinal_cord_symptom,brainstem_symptom,eye_symptom,supratentorial_symptom,other_symptoms,pyramidal,cerebellar,brainstem,sensory,bowel_and_bladder,visual_function,cerebral_functions,ambulation
0,106,2.5,1,1,17,23,0,1,0,1,0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
1,0,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,16,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,21,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,35,2.5,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


In [6]:
# Mice forest imputation
mice_missing = missing_data.copy()

kernel = mf.ImputationKernel(
  data=mice_missing,
  save_all_iterations=True,
  random_state=1343)

kernel.mice(3,verbose=True)
mice_missing = kernel.complete_data(dataset=0, inplace=False)
mice_missing.isnull().sum()

Initialized logger with name mice 1-3
Dataset 0
1  | cerebral_functions | pyramidal | cerebellar | brainstem | sensory | bowel_and_bladder | visual_function | ambulation
2  | cerebral_functions | pyramidal | cerebellar | brainstem | sensory | bowel_and_bladder | visual_function | ambulation
3  | cerebral_functions | pyramidal | cerebellar | brainstem | sensory | bowel_and_bladder | visual_function | ambulation


week_from_baseline                0
edss_as_evaluated_by_clinician    0
sex                               0
ms_in_pediatric_age               0
age_at_onset                      0
age_at_baseline                   0
spinal_cord_symptom               0
brainstem_symptom                 0
eye_symptom                       0
supratentorial_symptom            0
other_symptoms                    0
pyramidal                         0
cerebellar                        0
brainstem                         0
sensory                           0
bowel_and_bladder                 0
visual_function                   0
cerebral_functions                0
ambulation                        0
dtype: int64

In [12]:
#MissForest imputation
missforest_missing = missing_data.copy()
imputer = MissForest(max_depth=15) #miss forest
missforest_imputed = imputer.fit_transform(missforest_missing)
missforest_imputed = pd.DataFrame(missforest_imputed, columns = missforest_missing.columns)

int_columns = [0,2,3,4,5,6,7,8,9,10]
int_features = missforest_imputed.iloc[:,int_columns]
data_int_features = int_features.apply(lambda col: col.astype(int) if col.dtype == float else col)
float_features = [col_idx for col_idx, col in enumerate(missforest_imputed.columns) if col_idx not in int_columns]
data_float_features = missforest_imputed.iloc[:,float_features]
missforest_imputed = pd.concat([data_int_features, data_float_features], axis=1)
selected_columns = ["week_from_baseline", "edss_as_evaluated_by_clinician"]
selected_columns.extend(missforest_imputed.columns[1:10])
selected_columns.extend(missforest_imputed.columns[11:])
missforest_imputed = missforest_imputed[selected_columns]
missforest_imputed[missforest_imputed.columns[11:]] = missforest_imputed[missforest_imputed.columns[11:]].round()
imputer

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 0


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 1


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

...

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 3


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 4


MissForest(max_depth=15)

In [16]:
#KNN imputation K=7
Knn_missing = missing_data.copy()
impute = KNNImputer(weights = "distance")
KNN_imputed = imputer.fit_transform(Knn_missing)
KNN_imputed = pd.DataFrame(KNN_imputed, columns = missing_data.columns)

int_columns = [0,2,3,4,5,6,7,8,9,10]
int_features = KNN_imputed.iloc[:,int_columns]
data_int_features = int_features.apply(lambda col: col.astype(int) if col.dtype == float else col)
float_features = [col_idx for col_idx, col in enumerate(KNN_imputed.columns) if col_idx not in int_columns]
data_float_features = KNN_imputed.iloc[:,float_features]

KNN_imputed = pd.concat([data_int_features, data_float_features], axis=1)
selected_columns = ["week_from_baseline", "edss_as_evaluated_by_clinician"]
selected_columns.extend(KNN_imputed.columns[1:10])
selected_columns.extend(KNN_imputed.columns[11:])

KNN_imputed = KNN_imputed[selected_columns]
KNN_imputed[KNN_imputed.columns[11:]] = KNN_imputed[KNN_imputed.columns[11:]].round()
KNN_imputed.head()

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 0


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 1


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

...

c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 3


c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
c:\users\mahin vazifehdan\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 an

Iteration: 4


,week_from_baseline,edss_as_evaluated_by_clinician,sex,ms_in_pediatric_age,age_at_onset,age_at_baseline,spinal_cord_symptom,brainstem_symptom,eye_symptom,supratentorial_symptom,other_symptoms,pyramidal,cerebellar,brainstem,sensory,bowel_and_bladder,visual_function,cerebral_functions,ambulation
0,106,2.5,1,1,17,23,0,1,0,1,0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
1,0,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,16,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,21,2.0,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,35,2.5,0,0,26,28,1,1,0,1,0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


In [18]:
# Compare imputed data with real values
mse_kmeans = mean_squared_error(completed_data.values.flatten(), kmean_missing.values.flatten())
mse_mice = mean_squared_error(completed_data.values.flatten(), mice_missing.values.flatten())
mse_missfo = mean_squared_error(completed_data.values.flatten(), missforest_imputed.values.flatten())
mse_knn = mean_squared_error(completed_data.values.flatten(), KNN_imputed.values.flatten())
print("Kmean Error:", mse_kmeans.round(4))
print("Mice Error:", mse_mice.round(4))
print("Missforest Error:", mse_missfo.round(4)) 
print("KNN Error:", mse_knn.round(4)) # The Best

Kmean Error: 0.0323
Mice Error: 0.0056
Missforest Error: 0.0045
KNN Error: 0.004
